This file useses the purchases of only the 2021 FA, because we need prod. char

In [144]:
%reset -f

In [ ]:
!pip install pandasgui
!pip install pyblp 
!pip install unidecode

In [145]:
import os
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib.ticker import MaxNLocator
import seaborn as sns
from pandasgui import show
import socket
import numpy as np
from unidecode import unidecode
import unicodedata
import re
from difflib import SequenceMatcher
import pyblp
from scipy.stats import chi2


[notice] A new release of pip is available: 24.1.2 -> 24.3.1
[notice] To update, run: C:\Users\lucas\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 24.1.2 -> 24.3.1
[notice] To update, run: C:\Users\lucas\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 24.1.2 -> 24.3.1
[notice] To update, run: C:\Users\lucas\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


Functions

In [146]:
def calculate_market_shares_with_outside_good(df_transac, share_adjustment = True):
    """
    Calculate market shares using market-specific N_i from the DataFrame
    """
    # Group by product (j) and market (t) to count purchases
    product_counts = df_transac.groupby(['product_ids', 'market_ids']).size().reset_index(name='purchases')
    
    # Get N_i for each market (taking first occurrence since it's constant within market)
    market_populations = df_transac.groupby('market_ids')['N_i'].first().reset_index()
    
    # Merge N_i with product counts
    product_counts = product_counts.merge(market_populations, on='market_ids')
    
    # Calculate market shares
    if share_adjustment: 
        product_counts['shares'] = np.minimum(10*product_counts['purchases'] / product_counts['N_i'], 0.5)
    else: 
        product_counts['shares'] = product_counts['purchases'] / product_counts['N_i']
    
    product_counts['shares_real'] =product_counts['purchases'] / product_counts['N_i']
                                               
    x_vars = [col for col in df_transac.columns if col.startswith('x')] #list of variables starting with x 
    
    # Get product characteristics
    product_chars = df_transac.groupby(['product_ids', 'market_ids']).agg(
        {var: 'first' for var in x_vars} | {'prices': 'first', 'model_id': 'first'}
    ).reset_index()
    
    # Merge market shares with product characteristics
    final_df = product_chars.merge(product_counts[['product_ids', 'market_ids', 'shares', 'shares_real']], on=['product_ids', 'market_ids'])
    
    # Calculate outside good share for each market
    market_shares = final_df.groupby('market_ids')['shares'].sum()
    outside_shares = 1 - market_shares
    market_shares_real = final_df.groupby('market_ids')['shares_real'].sum()
    outside_shares_real = 1 - market_shares_real
    
    
    # Create outside good rows
    markets = df_transac['market_ids'].unique()
    outside_goods = pd.DataFrame()
    outside_goods['market_ids'] = markets
    outside_goods['product_ids'] = 0

    for var in x_vars:
        if pd.api.types.is_numeric_dtype(df_transac[var]):
            outside_goods[var] = 0  # Set numeric variables to 0
        else:
            outside_goods[var] = '-'  # Set categorical variables to '-'
    
    outside_goods['prices'] = 0
    outside_goods['shares'] = [outside_shares[t] for t in markets]
    outside_goods['shares_real'] = [outside_shares_real[t] for t in markets]

    # Combine outside goods with other products
    final_df = pd.concat([final_df, outside_goods])
    
    # Sort by market and product ID
    final_df = final_df.sort_values(['market_ids', 'product_ids'])
    
    final_df['model_id'] = final_df['model_id'] + 1 
    final_df['model_id'] = final_df['model_id'].fillna(0)
    return final_df

def normalize_shares(group):
    """
    Function to normalize shares for each market
    """
    # Fix shares that are not negative
    total_shares = group[group['shares'] > 0.01]['shares'].sum()
    scaling_factor = 0.99 - group[group['shares'] == 0.01]['shares'].sum()
    if total_shares > 0:  # Avoid division by zero
        group.loc[group['shares'] > 0.01, 'shares'] *= scaling_factor / total_shares
    return group

In [302]:
car_path = os.path.join('..', 'interm_data', 'yearly_data', 'Transacciones', 'transac_with_chars.csv')
print(os.path.abspath(car_path))
df = pd.read_csv(car_path, encoding = 'latin1')

c:\Users\lucas\OneDrive - Yale University\Documents\GitHub\2nd-year-paper\interm_data\yearly_data\Transacciones\transac_with_chars.csv


## data cleaning

1. select vars that could be relevant
2. rename prod. char as x1, x2.. and do other renaming 
3.  to reduce fixed effects if a variable is categorical and has more than 5 values we remove it. 

In [303]:
#### 1. variable selection 
char_list  = ["Motor - Combustible", "Motor - Cilindrada",
    "TransmisiÃ³n y chasis - Motor - tracciÃ³n", "TransmisiÃ³n y chasis - TransmisiÃ³n",
    "TransmisiÃ³n y chasis - SuspensiÃ³n trasera", "Medidas y capacidades - Largo",
    "Medidas y capacidades - Alto", "Medidas y capacidades - Distancia entre ejes",
    "Confort - Aire acondicionado", "Confort - Tapizados", "Confort - Cierre de puertas", 
    "Confort - Vidrios (del. - tras.)", "Confort - Espejos exteriores", "Confort - Espejo interior",
    "Confort - Faros delanteros", "Confort - Faros antiniebla", "Confort - Computadora de a bordo",
    "Confort - DirecciÃ³n asistida", "Seguridad - Airbags", "Tipo de Producto"]

model_vars = ["IDProductoCM",  "Modelo",  "Marca", 'simplified_model',
            "Precio Unitario", "Rut Unidad de Compra", "RegiÃ³n Unidad de Compra", "Sector",
            "year_df", "Nro LicitaciÃ³n PÃºblica", "Fecha EnvÃ­o OC", "Cantidad", "Rut Proveedor",
             "Nombre Proveedor Sucursal", ]
df = df[model_vars + char_list]


### 2. rename vars 
for n, col in enumerate(char_list, start=1):  # Start numbering from 1
    df.rename(columns={col: f'x{n}'}, inplace=True)

df = df.rename(columns={
    'Fecha EnvÃ­o OC': 'Fecha',
    'RegiÃ³n Unidad de Compra': 'Region', 
    'IDProductoCM': 'product_ids',
})

### 3. drop vars to reduce number of parameters to estimate
for n, col in enumerate(char_list, start=1):  # Start numbering from 1
    print(f"number of unique values in {f'x{n}'}: {df[f'x{n}'].nunique()}")


drop_list = ['x4', 'x5', 'x10', 'x12', 'x13','x15',  'x16', 'x17', 'x18', 'x19']
df = df.drop(drop_list, axis=1)

number of unique values in x1: 4
number of unique values in x2: 65
number of unique values in x3: 3
number of unique values in x4: 17
number of unique values in x5: 39
number of unique values in x6: 132
number of unique values in x7: 119
number of unique values in x8: 93
number of unique values in x9: 5
number of unique values in x10: 8
number of unique values in x11: 4
number of unique values in x12: 8
number of unique values in x13: 21
number of unique values in x14: 2
number of unique values in x15: 39
number of unique values in x16: 6
number of unique values in x17: 1
number of unique values in x18: 7
number of unique values in x19: 12
number of unique values in x20: 8


1. Create month-year 
2. create markets and number of consumers
3. create consumer group

In [304]:
#1. create month-year 
df['Fecha'] = pd.to_datetime(df['Fecha'], errors='coerce')
df['month'] = df['Fecha'].dt.month
df['year'] = df['Fecha'].dt.year
df.drop('Fecha', axis=1, inplace=True)

#2. define the market and the number of consumers in the market 
df['semester'] = ((df['month'] - 1) // 6) + 1
df['market_ids'] = df.groupby(['year', 'semester', 'Region']).ngroup()
print(df['market_ids'].nunique())

region_counts = df.groupby('Region')['market_ids'].nunique()
df['N_i'] = df['Region'].map(region_counts)

## 3. create consumer groups based on zone-sector
#print(df['Region'].unique())
Norte = ['Arica y Parinacota', 'TarapacÃ\x83Â\x83Ã\x82Â¡', 'Antofagasta', 'Atacama', 'Coquimbo']
Centro = ["Lib. Gral. Bdo. O'Higgins", 'Maule', 'Metropolitana', 'ValparaÃ\x83Â\x83Ã\x82Â\xadso' ]
Sur = ['BÃ\x83Â\x83Ã\x82Â\xado-BÃ\x83Â\x83Ã\x82Â\xado', 'Los RÃ\x83Â\x83Ã\x82Â\xados', 'Ã\x83Â\x83Ã\x82Â\x91uble',
    'Los Lagos', 'AraucanÃ\x83Â\x83Ã\x82Â\xada', 'Magallanes y AntÃ\x83Â\x83Ã\x82Â¡rtica', 'AysÃ\x83Â\x83Ã\x82Â©n']


def get_zone(region):
    if region in Norte:
        return 'Norte'
    elif region in Centro:
        return 'Centro'
    elif region in Sur:
        return 'Sur'
    else:
        return 'Other'  # For any regions not in the lists
df['Zona'] = df['Region'].apply(get_zone)
#df['k'] = df.groupby(['Sector', 'Zona']).ngroup()
df['k'] = df.groupby('Sector').ngroup()

217


0. generate unique identifier by simplified model 
1. convert to numeric values and drop chars with NaNs 
2. drop columns with NaNs since they can not be used.


In [305]:
#0. 
# Create a dictionary mapping unique models to IDs
model_to_id = {model: idx for idx, model in enumerate(df['simplified_model'].unique())}

# Create new column with the numeric IDs
df['model_id'] = df['simplified_model'].map(model_to_id)


# 1. convert to numeric values and drop chars with NaNs 
def extract_number(value):
    if pd.isna(value) or value == 'N/D':
        return np.nan
    # Extract numbers using string methods
    if isinstance(value, str):
        # Find first sequence of numbers (including decimals)
        numbers = ''.join(c for c in value if c.isdigit() or c == '.')
        return float(numbers) if numbers else value
    return value

for column in df.columns:
    try:
        df[column] = df[column].apply(extract_number)
    except:
        continue # If conversion fails, keep the original values


## 2. drop rows with NaNs
x_columns = [col for col in df.columns if col.startswith('x')]
x_columns_with_missing = [col for col in x_columns if df[col].isna().any()] #cols start with x and have NaNs
non_x_columns = [col for col in df.columns if not col.startswith('x')] # Get non-x columns 
df = df.drop(columns=x_columns_with_missing) # Drop x-columns with missing values

remaining_x_columns = [col for col in df.columns if col.startswith('x')]
new_x_names = [f'x{i+1}' for i in range(len(remaining_x_columns))] #rename x cols so that they dont skip numbers 
rename_dict = dict(zip(remaining_x_columns, new_x_names))
df = df.rename(columns=rename_dict) # Rename only the x-columns

In [306]:
df['prices'] = df['Precio Unitario'].replace(r',.*', '', regex=True) #remove commas from numbers

columns_to_convert = ['prices'] 

# Convert the specified columns to numeric
df[columns_to_convert] = df[columns_to_convert].apply(pd.to_numeric, errors='coerce')





only keep product categories with more than 100 obs 

In [307]:
print(df.value_counts('x6'))
# Look at value counts and drop categories with less than 100 occurrences
x6_counts = df['x6'].value_counts()
print("Categories and their counts:")
for cat, count in x6_counts.items():
    print(f"{cat}: {count}")

# Keep only categories with >= 100 occurrences 
valid_categories = x6_counts[x6_counts >= 100].index
df['x6'] = df['x6'].apply(lambda x: x if x in valid_categories else None)

x6
CAMIONETA          2183
SUV                 894
MINIBUS             364
SEDÃN              232
CAMIÃN LIVIANO      97
FURGÃN              88
HATCHBACK            35
CARGO                 1
Name: count, dtype: int64
Categories and their counts:
CAMIONETA: 2183
SUV: 894
MINIBUS: 364
SEDÃN: 232
CAMIÃN LIVIANO: 97
FURGÃN: 88
HATCHBACK: 35
CARGO: 1


In [308]:
product_data = calculate_market_shares_with_outside_good(df, share_adjustment=False)

check that shares 1. are positive and 2. sum upp to less than 1. 

In [309]:
###1. Check for negative shares
# percentage of neg shares. (if share is negative replace by 2%)
negative_shares_percentage = (product_data['shares'] < 0).mean() * 100
print(f"Percentage of negative shares: {negative_shares_percentage:.2f}%")
product_data['shares'] = product_data['shares'].apply(lambda x: 0.02 if x < 0.02 else x)

#shares have to sum up to 1%, normalize  by market_ids
product_data['shares_sum'] = product_data.groupby('market_ids')['shares'].transform('sum')
product_data['shares'] = product_data['shares'] / (product_data['shares_sum']+0.05) # add 0.05 since shares have to sum up to less than 1. 

# 3. check that outside shares are not too high and that there are no negative shares
outside_good_shares = product_data[product_data['product_ids'] == 0]['shares'].describe()
print(outside_good_shares)
negative_shares_percentage = (product_data['shares'] < 0).mean() * 100
print(f"Percentage of negative shares: {negative_shares_percentage:.2f}%")

# 4. check that no share sum up to more than 1 
sum_market_shares = product_data.groupby('market_ids')['shares'].sum()
markets_with_high_shares = sum_market_shares[(sum_market_shares > 1) ]
print(markets_with_high_shares)



Percentage of negative shares: 3.17%
count    217.000000
mean       0.313687
std        0.331115
min        0.001582
25%        0.011676
50%        0.136054
75%        0.634921
max        0.884354
Name: shares, dtype: float64
Percentage of negative shares: 0.00%
Series([], Name: shares, dtype: float64)


In [310]:
df_store = df.copy()
product_data_store = product_data.copy() # to have a copy of the original data

## Py-BLP with micro-moments. 

product_data is the data with the prices/shares/prod char,  in the documentation called product data
agent_data is the data with the demographics, in our case it has the distribution of demographics. 

\begin{align*}
    &E[\text{i purchases new vehicle} \mid \{\bar{y}_i < \bar{y}_1\}], \\
    &E[\text{i purchases new vehicle} \mid \{\bar{y}_1 \leq \bar{y}_i < \bar{y}_2\}], \\
    &E[\text{i purchases new vehicle} \mid \{\bar{y}_i \geq \bar{y}_2\}];
\end{align*}

\begin{align*}
    &E[f_{si} \mid \{\text{i purchases a minivan}\}], \\
    &E[f_{si} \mid \{\text{i purchases a station wagon}\}], \\
    &E[f_{si} \mid \{\text{i purchases a sport-utility}\}], \\
    &E[f_{si} \mid \{\text{i purchases a full-size van}\}].
\end{align*}


For each market create one obs. for each local agency in the market. 

### Loading data

1. product_data needs to have instruments explicitly defined as demand_instruments{i} variables? I am not sure because chat gpt at some pooint said that X1 variables automatically become instruments. 

2. the best way of dealing with categorical variables is to create lists of dummies, for example if you have k (consumer type) that takes values from 0 to 10 it is better to create k1 to k11. This makes it easier to create the micro-parts using the agent formulation. 
The same applies with product characteristics that are dummies and will be used to create micro-parts. 

In [323]:
df = df_store.copy() #restore the original data
product_data = product_data_store.copy() #restore the original data 
product_data.drop(columns = ['product_ids'], inplace =  True )
product_data.rename(columns = {'model_id': 'product_ids'}, inplace = True)

In [324]:
# create dummies for k 
k_dummies = pd.get_dummies(df['k'], prefix='k', dtype=int)
df = pd.concat([df, k_dummies], axis=1)
k_list = k_dummies.columns.tolist()
#k_list.remove('k_6')
#k_list.remove('k_5')
print(k_list)


['k_0', 'k_1', 'k_2', 'k_3', 'k_4', 'k_5', 'k_6']


transform x6 to dummy vars x7 to xN 

In [325]:
# Get the unique categories from one dataset (assuming the categories are the same in both datasets)
categories = product_data['x6'].unique()

# Step 1: Create dummies for product_data
dummies_product = pd.get_dummies(product_data['x6'], dtype=int)
dummies_product = dummies_product.reindex(columns=categories, fill_value=0)
dummies_product.columns = [f"x{i}" for i in range(7, 7 + len(categories))]
product_data = pd.concat([product_data, dummies_product], axis=1)

# Step 2: Create dummies for df
dummies_df = pd.get_dummies(df['x6'], dtype=int)
dummies_df = dummies_df.reindex(columns=categories, fill_value=0)
dummies_df.columns = [f"x{i}" for i in range(7, 7 + len(categories))]
df = pd.concat([df, dummies_df], axis=1)
car_type_dummies = ['x8', 'x9',  'x11', 'x12'] #do not include 'x10' because it was causing collinearity issues. 

print(car_type_dummies)
product_data.drop('x7', axis= 1,  inplace = True) #is a dummy for missing values 

#car_type_dummies = ['x8', 'x9', 'x10']

['x8', 'x9', 'x11', 'x12']


generate agent data 

In [326]:
def create_agent_data(product_data, df, N=100):
    # Validate inputs
    if 'market_ids' not in product_data.columns or 'market_ids' not in df.columns:
        raise ValueError("Both product_data and df must have a 'market_ids' column.")
    if 'k' not in df.columns:
        raise ValueError("The 'df' dataframe must contain a 'k' column.")
    
    # Get unique market IDs
    market_ids = product_data['market_ids'].unique()
    
    # Create empty lists to store data
    agent_data_list = []
    
    # Generate data for each market
    for market_id in market_ids:
        # Generate market IDs and weights
        market_ids_i = np.full(N, market_id)
        weights_i = np.full(N, 1 / N)
        
        # Generate nodes from chi-square(3) distribution
        nodes_i = chi2.rvs(df=3, size=(N, 11))
        
        # Get k distribution for this market
        market_k = df[df['market_ids'] == market_id]['k'].value_counts()
        if market_k.empty:
            raise ValueError(f"No 'k' data available for market_id {market_id}.")
        probabilities = market_k / market_k.sum()
        
        # Generate k values based on the distribution
        k_i = np.random.choice(market_k.index, size=N, p=probabilities)
        
        # Combine all into a single DataFrame for this market
        market_data = pd.DataFrame({
            'market_ids': market_ids_i,
            'weights': weights_i,
            'k': k_i
        })
        
        for i in range(11):
            market_data[f'nodes{i}'] = nodes_i[:, i]
        
        # Append to the main list
        agent_data_list.append(market_data)
    
    # Concatenate all market data
    agent_data = pd.concat(agent_data_list, ignore_index=True)
    
    # Create dummy variables for k
    agent_data = pd.concat([agent_data, pd.get_dummies(agent_data['k'], prefix='k')], axis=1)
    
    return agent_data


# Create the agent_data DataFrame
agent_data = create_agent_data(product_data, df)

# Sort by market_ids for cleaner output
agent_data = agent_data.sort_values('market_ids').reset_index(drop=True)
agent_data


,market_ids,weights,k,nodes0,nodes1,nodes2,nodes3,nodes4,nodes5,nodes6,...,nodes8,nodes9,nodes10,k_0,k_1,k_2,k_3,k_4,k_5,k_6
0,0,0.01,3,1.472936,1.749826,0.277410,4.465586,1.221116,3.420080,0.745126,...,0.030344,1.625646,1.490579,False,False,False,True,False,False,False
1,0,0.01,3,1.874038,1.091387,0.253513,0.259205,0.938425,3.946507,12.610528,...,4.912883,3.649473,5.434057,False,False,False,True,False,False,False
2,0,0.01,3,1.790057,8.055982,3.225583,5.863133,2.433661,3.220127,3.922476,...,1.701982,1.774889,4.423523,False,False,False,True,False,False,False
3,0,0.01,3,0.764926,8.433982,2.123790,1.666927,1.836264,0.398224,3.423341,...,1.572102,2.059631,9.043988,False,False,False,True,False,False,False
4,0,0.01,3,1.636307,2.174726,0.167153,2.912481,2.724134,1.631412,2.021295,...,1.216795,1.971932,2.458951,False,False,False,True,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
21695,216,0.01,3,3.307148,1.473756,4.700954,1.597808,2.243162,8.936660,1.304072,...,2.393040,4.969297,2.975508,False,False,False,True,False,False,False
21696,216,0.01,3,5.427338,1.826279,11.441033,1.572075,5.546083,2.503798,7.851869,...,1.489206,1.814205,9.722421,False,False,False,True,False,False,False
21697,216,0.01,3,3.157814,3.906189,5.198243,1.806855,1.136933,12.843746,8.739671,...,0.188150,3.663534,2.180201,False,False,False,True,False,False,False
21698,216,0.01,1,3.328665,1.811795,3.434160,3.859996,2.931855,2.477535,4.419991,...,1.292895,0.700553,2.563057,False,True,False,False,False,False,False


### setting up the problem 

In [327]:
#product formulation incorporates X1, X2 and X3. 
# X1: linear parameters of the utility function 
# X2: non-linear parameters of the utility function 
# X3: linear parameters of the cost function
#X1_formulation = pyblp.Formulation('0 + prices ', absorb='C(x1) + C(x4) + C(x5) ') # when using absorb I got this error "AbsorptionError: A fixed effect absorption procedure failed to properly absorb fixed effects. Consider configuring absorption options or choosing a different absorption method. For information about absorption options and defaults, refer to the PyHDFE package's documentation. Exception encountered: 'zero-size array to reduction operation maximum which has no identity'."
K1 = 5 # vars without random coefficients 
K2 = len(car_type_dummies) + 2  # vars with random coefficients, the 1 is compulsory 
D = len(k_list) 


X1_formulation = pyblp.Formulation('0 + prices + C(x1) + x2 + x3 + C(x4) + C(x5) + '+ ' + '.join(car_type_dummies))
X1_formulation = pyblp.Formulation('0 + prices + x2 + x3 + '+ ' + '.join(car_type_dummies)) #removed categorical variables to reduce the dimensionality of the problem. 


#X2_formulation = pyblp.Formulation('1 + prices + C(x6)') # the problem was that x6 was empty for the outside alternative
X2_formulation = pyblp.Formulation('prices + ' + ' + '.join(car_type_dummies)
)
#X2_formulation = pyblp.Formulation('prices + x8 + x9 +  x11' )
#X2_formulation = pyblp.Formulation('prices + x2 + x3' )
product_formulations = (X1_formulation, X2_formulation ) 
product_formulations

(prices + x2 + x3 + x8 + x9 + x11 + x12, 1 + prices + x8 + x9 + x11 + x12)

In [328]:
agent_formulation = pyblp.Formulation('1 + C(k)') #C(k) takes k as a categorical variable, if one wants to include a formula, for example income/price then one uses I(income/prices) 

agent_formulation = pyblp.Formulation('1 + '+ ' + '.join(k_list)) 

#k_list2 = k_list.copy() 
#k_list2.remove('k_0')
#k_list2.remove('k_1')
#k_list2.remove('k_2')
#agent_formulation = pyblp.Formulation('1 + '+ ' + '.join(k_list2)) 
#agent_formulation = pyblp.Formulation('k_3 + k_4 + k_5 + k_6') 

agent_formulation

1 + k_0 + k_1 + k_2 + k_3 + k_4 + k_5 + k_6

In [329]:
problem = pyblp.Problem(product_formulations, product_data, agent_formulation, agent_data)


Initializing the problem ...
Initialized the problem after 00:00:00.

Dimensions:
 T    N      I     K1    K2    D    MD 
---  ----  -----  ----  ----  ---  ----
217  2936  21700   7     6     8    6  

Formulations:
       Column Indices:           0         1          2          3          4          5          6          7    
-----------------------------  ------  ---------  ---------  ---------  ---------  ---------  ---------  ---------
 X1: Linear Characteristics    prices     x2         x3         x8         x9         x11        x12              
X2: Nonlinear Characteristics    1      prices       x8         x9         x11        x12                         
       d: Demographics           1     k_0[True]  k_1[True]  k_2[True]  k_3[True]  k_4[True]  k_5[True]  k_6[True]


observations 
1. absorbed cause multicollinearity issues in the vars, be careful when using it. 

### setting up micro-moments

In [330]:
micro_dataset = pyblp.MicroDataset(
    name="Purchases",
    observations=len(df),
    compute_weights=lambda t, p, a: np.ones((a.size, p.size)), # Equal weights for all observations 
    # if the microdata includes the outside good then it should be p.size + 1 
)
micro_dataset

Purchases: 3894 Observations in All Markets

construct the micro statistics. 

In [331]:
def create_micro_statistics(df, k_list, car_type_dummies):
    # Initialize lists to store results
    moments = []
    values = []
    
    # Loop over each k dummy variable
    for k_var in k_list:
        # Get observations where k_i = 1
        df = df.reset_index(drop=True)

        k_mask = df[k_var] == 1
        k_observations = df[k_mask]
        
        if len(k_observations) > 0:  # Only calculate if we have observations
            # Calculate average price conditional on k_i = 1
            avg_price_k = k_observations['prices'].mean()
            moments.append(f'E[price | {k_var}=1]')
            values.append(avg_price_k)
            
            # Loop over each car type dummy variable
            for car_dummy in car_type_dummies:
                # Calculate P(car_type=1 | k_i=1)
                car_cond_prob = k_observations[car_dummy].mean()
                moments.append(f'E[{car_dummy} | {k_var}=1]')
                values.append(car_cond_prob)
    
    # Create the micro_statistics DataFrame
    micro_statistics = pd.DataFrame({
        'value': values
    }, index=moments)
    
    return micro_statistics



micro_statistics = create_micro_statistics(df, k_list, car_type_dummies)


Each micro- moment has two parts. Let's exemplify with the case of the average price paid by consumers of group k = {a} where imagine that we created dummies and we have k{a} =1 for that group

1. The first part is the total numerator and is the total price paid by consumers of type {a}
p_k{a}_num = pyblp.MicroPart(
    name = 'E[p*( k{a} =1)]', 
    dataset = micro_dataset,
    compute_values = lambda t, p, a: np.outer(a.demographics[:, z], np.r_[0, p.X2[:, w]])
)

In this case z: is the position of the variable  k{a} in the agent formulation. and w: is the position of the product characteristic in this case x8 in X2_formulation in product formulations. 

2. the second part is the numerator and counts how many consumers of type {a} are in our data. 

k{a}_den = pyblp.MicroPart(
    name = 'E[ k{a} =1]', 
    dataset = micro_dataset,
    compute_values = lambda t, p, a: np.outer(a.demographics[:, z], np.r_[0, p.X2[:, 0]])
)
In this case z: is the position of the variable  k{a} in the agent formulation. and w= 0 because we are summing and the only element of the summation is whether the consumer is type k{a} and in X2_formulation the first element is 1 hence we are just multiplying by 1 which has no effect. 

Note that we are creating the moments in the opposite way to the documentation because their moments are E(k\type or price) whereas ours are E(type or price\k) 

In [332]:
# Dictionary to map x variables to their corresponding X2 indices
x_to_X2_index = {
    'x8': 2,
    'x9': 3,
    'x10': 4,
    'x11': 5,
    'x12': 6
}

# Initialize dictionary to store all micro parts
micro_parts = {}

 
for k_idx, k in enumerate(k_list):
    
    #create de numerator (in the paper is v_{3i})
    micro_parts[f'v_3{k}'] = pyblp.MicroPart(
            name=f'E[(k={k_idx})]',
            dataset=micro_dataset,
            compute_values=lambda t, p, a: np.outer(
                a.demographics[:, k_idx + 1], np.r_[0, p.X2[:, 0]])
        )
    
    #create deenominator of price moment (in the paper is v_{2i})
    micro_parts[f'v_2{k}'] = pyblp.MicroPart(
            name=f'E[p*(k={k_idx})]',
            dataset=micro_dataset,
            compute_values=lambda t, p, a: np.outer(
                a.demographics[:, k_idx + 1], np.r_[0, p.X2[:, 1]])
        )

    #create the numerators of type moments (in the paper is v_{1i\lambda})
    for x_var in car_type_dummies:
        
        X2_index = x_to_X2_index[x_var] # Get the corresponding X2 index
            
        micro_parts[f'v_3{k}{x_var}'] = pyblp.MicroPart(
            name=f'E[({x_var}=1)*(k={k_idx})]',
            dataset=micro_dataset,
            compute_values=lambda t, p, a: np.outer(
                a.demographics[:, k_idx + 1], np.r_[0, p.X2[:, X2_index]])
        )

# If you need them in separate variables instead of a dictionary:
locals().update(micro_parts)


In [333]:
compute_ratio = lambda v: v[0] / v[1]
compute_ratio_gradient = lambda v: [1 / v[1], -v[0] / v[1]**2]

In [334]:
micro_moments = [] 


for k_idx, k in enumerate(k_list):
    
    price_statistic =  micro_statistics.loc[f'E[price | {k}=1]', 'value']
    print(k_idx, k, price_statistic)
    

    price_moment = pyblp.MicroMoment(
        name = f'E[price|k={k_idx}]',
        value = price_statistic, 
        parts = [micro_parts[f'v_2{k}'], micro_parts[f'v_3{k}']], 
        compute_value = compute_ratio,
        compute_gradient = compute_ratio_gradient, 
    )
    micro_moments.append(price_moment)


    for x_var in car_type_dummies: 
        type_statistic = micro_statistics.loc[f'E[{x_var} | {k}=1]', 'value']
        print(x_var, type_statistic)


        type_moment = pyblp.MicroMoment(
            name = f'E[{x_var}|k={k_idx}]',
            value = type_statistic, 
            parts = [micro_parts[f'v_3{k}{x_var}'], micro_parts[f'v_3{k}']], 
            compute_value = compute_ratio,
            compute_gradient = compute_ratio_gradient, 
            )
        micro_moments.append(type_moment)



0 k_0 18046864.603833865
x8 0.18849840255591055
x9 0.4952076677316294
x11 0.10223642172523961
x12 0.11182108626198083
1 k_1 16679065.084867075
x8 0.35991820040899797
x9 0.48057259713701433
x11 0.04703476482617587
x12 0.07464212678936605
2 k_2 19737623.44827586
x8 0.5862068965517241
x9 0.13793103448275862
x11 0.1724137931034483
x12 0.06896551724137931
3 k_3 16443471.926708633
x8 0.16052158273381295
x9 0.6119604316546763
x11 0.06115107913669065
x12 0.09982014388489209
4 k_4 17707223.056338027
x8 0.3873239436619718
x9 0.5
x11 0.035211267605633804
x12 0.06338028169014084
5 k_5 17702596.98901099
x8 0.2967032967032967
x9 0.6703296703296703
x11 0.01098901098901099
x12 0.02197802197802198
6 k_6 17528794.0
x8 0.23076923076923078
x9 0.5213675213675214
x11 0.05982905982905983
x12 0.1794871794871795


In [335]:
print(micro_moments)
print(len(micro_moments))

[E[price|k=0]: +1.804686E+07 (E[p*(k=0)] on Purchases: 3894 Observations in All Markets; E[(k=0)] on Purchases: 3894 Observations in All Markets), E[x8|k=0]: +1.884984E-01 (E[(x8=1)*(k=0)] on Purchases: 3894 Observations in All Markets; E[(k=0)] on Purchases: 3894 Observations in All Markets), E[x9|k=0]: +4.952077E-01 (E[(x9=1)*(k=0)] on Purchases: 3894 Observations in All Markets; E[(k=0)] on Purchases: 3894 Observations in All Markets), E[x11|k=0]: +1.022364E-01 (E[(x11=1)*(k=0)] on Purchases: 3894 Observations in All Markets; E[(k=0)] on Purchases: 3894 Observations in All Markets), E[x12|k=0]: +1.118211E-01 (E[(x12=1)*(k=0)] on Purchases: 3894 Observations in All Markets; E[(k=0)] on Purchases: 3894 Observations in All Markets), E[price|k=1]: +1.667907E+07 (E[p*(k=1)] on Purchases: 3894 Observations in All Markets; E[(k=1)] on Purchases: 3894 Observations in All Markets), E[x8|k=1]: +3.599182E-01 (E[(x8=1)*(k=1)] on Purchases: 3894 Observations in All Markets; E[(k=1)] on Purchases

### solving the problem

the dimensions of sigma are the same as X2
the dimensionn of pi is K2xD where in the tutorial K2 = dim(X2 formulation) and D is the number of demographic vars. 


In [338]:
K2 = 6
D = 8
initial_sigma = np.eye(K2) 
initial_pi = np.eye(K2, D)


sigma_lb = np.zeros((K2, K2))
sigma_up = np.eye(K2)*10 
sigma_bounds = (sigma_lb, sigma_up)

pi_lb = np.zeros((K2, D)) 
pi_up = np.eye(K2, D)*10
pi_bounds = (pi_lb, pi_up)


In [343]:
results = problem.solve(
    sigma=initial_sigma,
    sigma_bounds=sigma_bounds,
    pi=initial_pi,
    pi_bounds = pi_bounds,
    optimization=pyblp.Optimization('bfgs', {'gtol': 1e-4}),
    iteration=pyblp.Iteration('squarem', {'atol': 1e-13}),
    micro_moments=micro_moments,
)
results

Solving the problem ...

Micro Moments:
  Observed        Moment           Part         Dataset   Observations  Markets
-------------  ------------  ----------------  ---------  ------------  -------
+1.804686E+07  E[price|k=0]     E[p*(k=0)]     Purchases      3894        All  
                                 E[(k=0)]      Purchases      3894        All  
+1.884984E-01   E[x8|k=0]    E[(x8=1)*(k=0)]   Purchases      3894        All  
                                 E[(k=0)]      Purchases      3894        All  
+4.952077E-01   E[x9|k=0]    E[(x9=1)*(k=0)]   Purchases      3894        All  
                                 E[(k=0)]      Purchases      3894        All  
+1.022364E-01   E[x11|k=0]   E[(x11=1)*(k=0)]  Purchases      3894        All  
                                 E[(k=0)]      Purchases      3894        All  
+1.118211E-01   E[x12|k=0]   E[(x12=1)*(k=0)]  Purchases      3894        All  
                                 E[(k=0)]      Purchases      3894        All  


ValueError: In market 0, micro moment part 'E[p*(k=0)] on Purchases: 3894 Observations in All Markets' returned an array of shape (100, 8) from compute_values, which is not (100, 7), the shape of the array returned by its dataset's compute_weights.

In [183]:
show(product_data)

PandasGUI INFO — pandasgui.gui — Opening PandasGUI


In [221]:
show(df)

PandasGUI INFO — pandasgui.gui — Opening PandasGUI
